# Gremlin Workshop - Section 05: Advanced Gremlin Query Patterns

In this section, we show you some more advanced Gremlin queries. There's virtually an infinite number of ways you can chain Gremlin steps together to build a traversal.  In this next notebook, we want to convey some of the more common query patterns and use cases.  In the process, we will introduce a few of the remaining Gremlin steps.

## Graph Model

As we have in previous notebooks, we've included here a copy of the graph data model for this dataset as reference to aid in writing your queries.

<img src="https://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/f3f89ef4607743429fb01ae23d983197/v1/workshop/images/datamodel-updated.png"/>


### 05.01 - Recommendation query

One of the more popular use cases for a graph database is for creating recommendations.  You may recall one of our earlier queries that was used to find Friends-of-Friends, or Co-Stars-of-Co-Stars.  Building recommendations can be very similar to that.

Start with a person with ID 'person378' and suggest 10 movie recommendations.
The movies should be highly rated and belong to the favourite genre of this person.

Consult the following documentation:
 - [`groupCount()`](http://tinkerpop.apache.org/docs/current/reference/#groupcount-step)
 - [`order().by()`](http://tinkerpop.apache.org/docs/3.3.2/reference/#order-step)


In [ ]:
%%gremlin

g.V('person378').out('rated').aggregate('watched').   //Find all movies watched/rated by person378
    out('genre').groupCount().
        order(local).by(values, desc).unfold().limit(1).   //Find the most popular genre of those movies
    select(keys).in('genre').
        has('averageRating', gt(8.0)).
        has('numVotes',gt(50000)).  //Find movies in that genre with averageRating > 8 and numVotes > 50000
    where(without('watched')).   //Filter out the movies person378 has already watched/rated
    order().by('averageRating', desc).   //Order the results by averageRating and return the top 10
    limit(10).values('title')

### 05.02 - The "Six Degrees of Kevin Bacon"

It wouldn't be a complete graph workshop if we didn't include the "Six Degrees of Kevin Bacon"! :)  If you're not familar with the concept, the rule of ["six degrees of separation"](https://en.wikipedia.org/wiki/Six_degrees_of_separation) postulates that everyone on the planet is connected through no more than six social relationships with anyone else on the planet.  That same concept has been extended within the realm of film, suggesting that all actors are no more than six degreess (across common co-stars and productions) connected to the actor Kevin Bacon.  (It is also a humorous joke because it seems like Kevin Bacon has performed in A LOT of movies!)

This query introduces you to `repeat().until()` and `path()` steps. These are a useful constuct to write multiple-hop queries with some terminating condition.  It is a form of recursion that is not easily done in other types of databases.

Consult the following documentation:
 - [`repeat().until()`](http://tinkerpop.apache.org/docs/3.3.2/reference/#repeat-step)
 - [`path()`](http://tinkerpop.apache.org/docs/3.3.2/reference/#path-step)
 - [`Additional reference`](http://kelvinlawrence.net/book/Gremlin-Graph-Guide.html#sp)

In the following query, let's see how many hops there are between James Earl Jones and Kevin Bacon.  

#### BONUS: Graph Visualizations

After running this query, note the new Graph tab that appears in the results pane.  Whenever returning a set of paths using Gremlin, the Neptune Workbench will render the results in a force-directed graph visualization.  You can learn more about how to use this feature in our [example Graph Visualization notebooks](../../02-Visualization).

In [ ]:
%%gremlin -p v,outv

g.V().hasLabel('Artist').
    has('name','James Earl Jones'). //starting with James Earl Jones (JEJ)
    repeat(
        in('actor','actress').
        out('actor','actress').   //traverse across all productions
        simplePath()).            //making sure not to return to JEJ
    until(has('name','Kevin Bacon')).  //until we reach Kevin Bacon
    path().by('name').by('title').limit(10)    //return the first 10 paths

Let's break down this query in a bit more detail.  We start with finding the vertex for James Earl Jones:

`g.V().hasLabel('Artist').has('name','James Earl Jones').`

We then use the `repeat()` step to create a recursive search for Kevin Bacon.  Within the `repeat()` step, we're traversing across the edge connecting an Artist to a production and then back to Artists.  We continue this pattern `until()` we get to a vertex with a name property of Kevin Bacon.  The `simplePath()` step in this tells us that we want to ignore cyclical paths.  This avoids a potential unending loop.

`repeat(in_('actor','actress').out('actor','actress').simplePath()).until(has('name','Kevin Bacon')).`

At this point, we have now traversed all possible outcomes coming from James Earl Jones and going to Kevin Bacon.  However, we want to output more than the end of our traversal at this point.  We want to output all vertices that we crossed in the process so that we can see the connections between James Earl Jones and Kevin Bacon.  We can do this using the `path()` step.  The `path()` step collects all traversed vertices and outputs chain of vertices composing a traversal.  Similar to steps like `select()` we can use the `by()` step to de-reference each vertex and output related properties.  In this case, we want the name or title property from each vertex.  When chaining `by()` statements together, they are executed in a round-robin fashion for all vertices in the path that are mapped to the `by()` steps.

`path().by('name').by('title').limit(10)`

Last, but not least, we want to limit our results.  There are probably thousands of paths between James Earl Jones and Kevin Bacon.  In this case, we only care that a path does exist and we may be interested in a few other possible paths.  We can limit the execution time of our query by only returning the first 10 results found.

_NOTE: Neptune executes traversals in a Breadth First execution pattern, by default.  So in many cases (but not all) we will see shorter paths returned first.  You can alter this behavior using [Neptune Query Hints](https://docs.aws.amazon.com/neptune/latest/userguide/gremlin-query-hints.html) to perform a Depth First traversal._

### 05.03 - Running average

Gremlin also provides a few mathematic steps. This includes things like `mean()`, `max()`, `min()`, and `math()`. The latter for simple arithmatic.

Consult the following documentation:
 - [`mean()`](http://tinkerpop.apache.org/docs/3.3.2/reference/#mean-step)
 - [`max()`](http://tinkerpop.apache.org/docs/3.4.4/reference/#max-step)
 - [`min()`](http://tinkerpop.apache.org/docs/3.4.4/reference/#min-step)
 - [`math()`](http://tinkerpop.apache.org/docs/3.4.4/reference/#math-step)
 
Let's try using the `mean()` step to take a running average of ratings for a particular Artist or production.  In the following two examples we look at the average rating "The Original Kings of Comedy".  In addition, we have a query looking at the average ratings of all movies directed by Roland Emmerich.

In [ ]:
%%gremlin

g.V().hasLabel('movie').has('title','The Original Kings of Comedy').
    inE('rated').
    values('rating').
    mean()

In [ ]:
%%gremlin

g.V().hasLabel('Artist').has('name', 'Roland Emmerich').
    in('director').
    inE('rated').
    values('rating').
    mean()

Yikes!  People must really not like Roland Emmerich's films.

### 05.04 - Another as() / select() example

Which is the most popular genre of movies that were directed by Martin Scorsese and where Leonardo DiCaprio acted in them.


In [ ]:
%%gremlin

g.V().hasLabel('Artist').
    has('name', 'Leonardo DiCaprio').as('actor').
    in('actor').as('movies').
    out('director').has('name', 'Martin Scorsese').
    select('movies').out('genre').
    groupCount().
    order(local).by(values, desc).unfold()

### 05.05 - Using project()

Similar to `select()` and `path()`, another step that can be used to format output is the `project()` step.  The `project()` step maps whatever it takes as an input to the subsequent parameters and associated `by()` statements.  This is a good step to use if you reach a point in the graph where you need to fork your traversal and output properties from further downstream edges and vertices.

Consult the following documentation:
 - [`project()`](http://tinkerpop.apache.org/docs/3.3.2/reference/#project-step)

Here's a good example. List the top 10 movies which have received highest average rating by the people. 

In [ ]:
%%gremlin

g.V().hasLabel('Artist').has('name','Carrie Fisher').
    in('actress').has('averageRating').
    project('movie', 'numratings').
        by('title').
        by('averageRating').
    order().
        by(select('numratings'), Order.desc)

### 05.06 - Social Rankings

Lastly, let's look at another example for proving connection based ranking.  In the following example, we will look at users and determine which of them are the most highly connected in our graph.


In [ ]:
%%gremlin

g.V().hasLabel('Person').
    project('firstName', 'lastName', 'numknows').
        by('firstName').
        by('lastName').
        by(__.out('knows').count()).
    order().
        by(select('numknows'), desc).
    limit(10)

### Review

In the previous notebook we looked at more advanced patterns for using Graph traversals to find interesting insights within our data.  These are just some of the different patterns that make graph useful.  We will use some of these patterns in later exercises in this workshop.

### End of Section 05 - Move onto the next notebook.